# Import packages

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns 

from sklearn.metrics import silhouette_score, silhouette_samples
import sklearn.metrics
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
from sklearn.mixture import GaussianMixture
from sklearn.metrics import accuracy_score, cohen_kappa_score, f1_score, log_loss
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification

import itertools

from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

import scipy

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from IPython.display import display
pd.options.display.max_columns = None

# Load data

In [2]:
df=pd.read_csv('/System/Volumes/Data/Users/Michael/Library/CloudStorage/iCloud Drive/MMA/MMA 831/allstate-purchase-prediction-challenge/train.csv')

# Check the data

In [3]:
# pandas_profiling.ProfileReport(df, check_correlation=False)

In [4]:
df.head(20)

,customer_ID,shopping_pt,record_type,day,time,state,location,group_size,homeowner,car_age,car_value,risk_factor,age_oldest,age_youngest,married_couple,C_previous,duration_previous,A,B,C,D,E,F,G,cost
0,10000000,1,0,0,08:35,IN,10001,2,0,2,g,3.0,46,42,1,1.0,2.0,1,0,2,2,1,2,2,633
1,10000000,2,0,0,08:38,IN,10001,2,0,2,g,3.0,46,42,1,1.0,2.0,1,0,2,2,1,2,1,630
2,10000000,3,0,0,08:38,IN,10001,2,0,2,g,3.0,46,42,1,1.0,2.0,1,0,2,2,1,2,1,630
3,10000000,4,0,0,08:39,IN,10001,2,0,2,g,3.0,46,42,1,1.0,2.0,1,0,2,2,1,2,1,630
4,10000000,5,0,0,11:55,IN,10001,2,0,2,g,3.0,46,42,1,1.0,2.0,1,0,2,2,1,2,1,630
5,10000000,6,0,0,11:57,IN,10001,2,0,2,g,3.0,46,42,1,1.0,2.0,1,0,2,2,1,2,1,638
6,10000000,7,0,0,11:58,IN,10001,2,0,2,g,3.0,46,42,1,1.0,2.0,1,0,2,2,1,2,1,638
7,10000000,8,0,0,12:03,IN,10001,2,0,2,g,3.0,46,42,1,1.0,2.0,1,0,2,2,1,2,1,638
8,10000000,9,1,0,12:07,IN,10001,2,0,2,g,3.0,46,42,1,1.0,2.0,1,0,2,2,1,2,1,634
9,10000005,1,0,3,08:56,NY,10006,1,0,10,e,4.0,28,28,0,3.0,13.0,1,1,3,3,1,0,2,755


# Slice the data and keep only the purchase record

In [5]:
df_purchased=df[df.record_type==1]
df_purchased.head()
df_purchased.info()

,customer_ID,shopping_pt,record_type,day,time,state,location,group_size,homeowner,car_age,car_value,risk_factor,age_oldest,age_youngest,married_couple,C_previous,duration_previous,A,B,C,D,E,F,G,cost
8,10000000,9,1,0,12:07,IN,10001,2,0,2,g,3.0,46,42,1,1.0,2.0,1,0,2,2,1,2,1,634
14,10000005,6,1,3,09:09,NY,10006,1,0,10,e,4.0,28,28,0,3.0,13.0,0,0,3,2,0,0,2,731
22,10000007,8,1,4,14:26,PA,10008,1,0,11,c,NaN,43,43,0,2.0,4.0,0,0,1,2,0,0,1,602
26,10000013,4,1,4,09:31,WV,10014,2,1,3,d,3.0,62,60,1,3.0,3.0,1,1,3,2,1,1,3,626
32,10000014,6,1,1,17:50,MO,10015,1,0,5,d,3.0,32,28,1,1.0,2.0,1,1,1,1,0,2,2,614


<class 'pandas.core.frame.DataFrame'>
Int64Index: 97009 entries, 8 to 665248
Data columns (total 25 columns):
customer_ID          97009 non-null int64
shopping_pt          97009 non-null int64
record_type          97009 non-null int64
day                  97009 non-null int64
time                 97009 non-null object
state                97009 non-null object
location             97009 non-null int64
group_size           97009 non-null int64
homeowner            97009 non-null int64
car_age              97009 non-null int64
car_value            97009 non-null object
risk_factor          62663 non-null float64
age_oldest           97009 non-null int64
age_youngest         97009 non-null int64
married_couple       97009 non-null int64
C_previous           96173 non-null float64
duration_previous    96173 non-null float64
A                    97009 non-null int64
B                    97009 non-null int64
C                    97009 non-null int64
D                    97009 non-null int64

# Create the purchase record table

In [6]:
list_product=["A","B","C","D","E","F","G"]
df_prducct=pd.DataFrame()
for i in list_product:
    df_prducct=pd.concat([df_prducct,pd.get_dummies(df_purchased[i],prefix=i)],1)
df_prducct

,A_0,A_1,A_2,B_0,B_1,C_1,C_2,C_3,C_4,D_1,D_2,D_3,E_0,E_1,F_0,F_1,F_2,F_3,G_1,G_2,G_3,G_4
8,0,1,0,1,0,0,1,0,0,0,1,0,0,1,0,0,1,0,1,0,0,0
14,1,0,0,1,0,0,0,1,0,0,1,0,1,0,1,0,0,0,0,1,0,0
22,1,0,0,1,0,1,0,0,0,0,1,0,1,0,1,0,0,0,1,0,0,0
26,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,1,0,0,0,0,1,0
32,0,1,0,0,1,1,0,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0
42,0,0,1,1,0,0,0,1,0,0,0,1,1,0,0,0,0,1,0,0,1,0
50,0,0,1,0,1,0,1,0,0,0,0,1,0,1,0,0,1,0,0,0,1,0
57,0,1,0,0,1,1,0,0,0,0,0,1,0,1,0,1,0,0,0,1,0,0
64,1,0,0,1,0,1,0,0,0,1,0,0,1,0,1,0,0,0,0,0,1,0
68,0,1,0,0,1,0,0,1,0,0,0,1,0,1,0,0,1,0,0,0,1,0


# Calculate the support

In [7]:
%time frequent_itemsets = apriori(df_prducct, min_support=0.001, use_colnames=True)
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets.head(10)
frequent_itemsets.tail(10)

CPU times: user 12.4 s, sys: 13 s, total: 25.4 s
Wall time: 23.6 s


,support,itemsets,length
0,0.220371,(A_0),1
1,0.614190,(A_1),1
2,0.165438,(A_2),1
3,0.523910,(B_0),1
4,0.476090,(B_1),1
5,0.298869,(C_1),1
6,0.210218,(C_2),1
7,0.391335,(C_3),1
8,0.099578,(C_4),1
9,0.130153,(D_1),1


,support,itemsets,length
7985,0.001155,"(A_2, C_3, G_3, D_3, F_2, E_0, B_1)",7
7986,0.001227,"(A_2, C_3, E_1, D_3, G_1, F_0, B_1)",7
7987,0.004701,"(A_2, C_3, E_1, G_2, D_3, F_0, B_1)",7
7988,0.001216,"(A_2, C_3, G_3, E_1, D_3, F_0, B_1)",7
7989,0.002484,"(F_1, A_2, C_3, G_3, E_1, D_3, B_1)",7
7990,0.003031,"(A_2, C_3, G_3, E_1, D_3, F_2, B_1)",7
7991,0.001165,"(A_2, F_3, C_3, E_1, G_2, D_3, B_1)",7
7992,0.001361,"(A_2, F_3, C_3, G_3, E_1, D_3, B_1)",7
7993,0.001278,"(A_2, C_4, E_1, G_2, D_3, F_0, B_1)",7
7994,0.001072,"(A_2, C_4, G_3, E_1, D_3, F_2, B_1)",7


In [8]:
frequent_itemsets[ (frequent_itemsets['length'] == 2) &
                   (frequent_itemsets['support'] >= 0.02) ]

,support,itemsets,length
22,0.154841,"(B_0, A_0)",2
23,0.065530,"(A_0, B_1)",2
24,0.105691,"(A_0, C_1)",2
25,0.051892,"(A_0, C_2)",2
26,0.054211,"(C_3, A_0)",2
28,0.052232,"(A_0, D_1)",2
29,0.061025,"(A_0, D_2)",2
30,0.107114,"(A_0, D_3)",2
31,0.214372,"(A_0, E_0)",2
33,0.192539,"(A_0, F_0)",2


# Calculate the support and confidence

In [9]:
%time rules = association_rules(frequent_itemsets, min_threshold=0.1)

CPU times: user 1.22 s, sys: 82.8 ms, total: 1.3 s
Wall time: 879 ms


In [10]:
rules["antecedent_len"] = rules["antecedents"].apply(lambda x: len(x))
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len
0,(B_0),(A_0),0.523910,0.220371,0.154841,0.295549,1.341143,0.039387,1.106719,1
1,(A_0),(B_0),0.220371,0.523910,0.154841,0.702638,1.341143,0.039387,1.601046,1
2,(A_0),(B_1),0.220371,0.476090,0.065530,0.297362,0.624592,-0.039387,0.745633,1
3,(B_1),(A_0),0.476090,0.220371,0.065530,0.137642,0.624592,-0.039387,0.904066,1
4,(A_0),(C_1),0.220371,0.298869,0.105691,0.479605,1.604733,0.039829,1.347306,1
5,(C_1),(A_0),0.298869,0.220371,0.105691,0.353637,1.604733,0.039829,1.206178,1
6,(A_0),(C_2),0.220371,0.210218,0.051892,0.235476,1.120152,0.005566,1.033038,1
7,(C_2),(A_0),0.210218,0.220371,0.051892,0.246849,1.120152,0.005566,1.035156,1
8,(C_3),(A_0),0.391335,0.220371,0.054211,0.138530,0.628619,-0.032028,0.904998,1
9,(A_0),(C_3),0.220371,0.391335,0.054211,0.246001,0.628619,-0.032028,0.807249,1


# Slice the data

In [11]:
rules[ (rules['antecedent_len'] >= 2) &
       (rules['confidence'] > 0.75) &
       (rules['lift'] > 1.2) ]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedent_len
369,"(A_0, D_1)",(B_0),0.052232,0.523910,0.040862,0.782317,1.493227,0.013497,2.187078,2
383,"(B_0, A_0)",(E_0),0.154841,0.537393,0.152934,0.987684,1.837916,0.069724,37.561152,2
390,"(B_0, A_0)",(F_0),0.154841,0.322094,0.138286,0.893083,2.772742,0.088413,6.340491,2
398,"(A_0, F_3)",(B_0),0.001917,0.523910,0.001484,0.774194,1.477722,0.000480,2.108398,2
436,"(A_0, B_1)",(E_0),0.065530,0.537393,0.061438,0.937549,1.744623,0.026222,7.407531,2
443,"(A_0, B_1)",(F_0),0.065530,0.322094,0.054253,0.827906,2.570388,0.033146,3.939168,2
462,"(A_0, D_1)",(C_1),0.052232,0.298869,0.049851,0.954411,3.193407,0.034240,15.379350,2
477,"(A_0, C_1)",(E_0),0.105691,0.537393,0.104434,0.988101,1.838692,0.047636,38.877911,2
484,"(A_0, C_1)",(F_0),0.105691,0.322094,0.095981,0.908124,2.819441,0.061938,7.378527,2
486,"(C_1, F_0)",(A_0),0.109866,0.220371,0.095981,0.873616,3.964291,0.071769,6.168732,2
